In [47]:
from typing import Tuple, List, Union, Callable
from pathlib import Path
import sys
import re

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms.v2 import ToTensor, Compose
from torchvision.datasets import VisionDataset

sys.path.append('..')
from libs import list_utils as lu
from libs import transforms as tsf
from libs import seglib, charters_htr
from model_htr import HTR_Model


In [30]:

# get line GT transcriptions.
# Input: JSON


class InferenceDataset( VisionDataset ):

    def __init__(self, img_path: Union[str,Path],
                 segmentation_data: Union[str,Path],
                 transform: Callable=None,
                 padding_style=None) -> None:
        """ A minimal dataset class for inference on a single charter (no transcription in the sample).

        Args:
            img_path (Union[Path,str]): charter image path

            segmentation_data (Union[Path, str]): segmentation metadata (XML or JSON)
            
            transform (Callable): Image transform.

            padding_style (str): How to pad the bounding box around the polygons, when 
                building the initial, raw dataset (before applying any transform):
                + 'median'= polygon's median value,
                + 'noise' = random noise,
                + 'zero'= 0-padding, 
                + None (default) = no padding, i.e. raw bounding box
        """

        trf = transform if transform else ToTensor()
        super().__init__(root, transform=trf )

        img_path = Path( img_path ) if type(img_path) is str else img_path
        segmentation_data = Path( segmentation_data ) if type(segmentation_data) is str else segmentation_data

        # extract line images: functions line_images_from_img_* return tuples (<line_img_hwc>: np.ndarray, <mask_hwc>: np.ndarray)
        line_extraction_func = seglib.line_images_from_img_json_files if segmentation_data.suffix == '.json' else seglib.line_images_from_img_xml_files

        line_padding_func = lambda x, m, channel_dim=2: x # by default, identity function
        if padding_style == 'noise':
            line_padding_func = tsf.bbox_noise_pad
        elif padding_style == 'median':
            line_padding_func = tsf.bbox_median_pad
        elif padding_style == 'zero':
            line_padding_func = tsf.bbox_zero_pad
        print(line_padding_func)

        self.data = []

        for (img_hwc, mask_hwc) in line_extraction_func( img_path, segmentation_data ):
            mask_hw = mask_hwc[:,:,0]
            self.data.append( { 'img': line_padding_func( img_hwc, mask_hw, channel_dim=2 ), #tsf.bbox_median_pad( img_hwc, mask_hw, channel_dim=2 ), 
                                'height':img_hwc.shape[0],
                                'width': img_hwc.shape[1],
                               } )

    def __getitem__(self, index: int):
        sample = self.data[index].copy()
        #logger.debug(f"type(sample['img'])={type(sample['img'])} with shape= {sample['img'].shape}" )
        return self.transform( sample )

    def __len__(self):
        return len(self.data)


In [34]:
# Get predicted lines, from img + segmentation file
root = Path('.')

collection_path = Path('/home/nicolas/tmp/data/fsdb_work/fsdb_sample/COLLECTIONS')
img_path = collection_path.joinpath('586c0318ce423a882dac8411c6c61978/869b3709dd67347d9d243bef546a8f2c/c87b4566ac286b5ad58fa4fe1c60eb2a.img.jpg')
segmentation_file_path = collection_path.joinpath('586c0318ce423a882dac8411c6c61978/869b3709dd67347d9d243bef546a8f2c/c87b4566ac286b5ad58fa4fe1c60eb2a.lines.gt.json')

dataset = InferenceDataset( img_path, segmentation_file_path,
                          transform = Compose([ ToTensor(),
                                                tsf.ResizeToHeight(128,2048),
                                                tsf.PadToWidth(2048),]),
                          padding_style='median')
model = HTR_Model.load('../model_koenigsfelden+nuremberg_letterbooks-2014.12.15.mlmodel')

<function bbox_median_pad at 0x735b6c1c2290>


In [20]:
Path.cwd()

PosixPath('/home/nicolas/professionnel/graz/htr/vre/ddpa_htr/notebooks')

In [35]:
predictions = []
for line, sample in enumerate(DataLoader(dataset,batch_size=1)):
    predicted_strings, line_scores = model.inference_task( sample['img'], sample['width'])
    predictions.append({'line_id': line, 'transcription': predicted_strings, 'scores': lu.flatten(line_scores.tolist())})

In [39]:
strings, scores = ( [ l[k] for l in predictions] for k in ('transcription', 'scores'))

([['Wir, Leupolt von gots gnaden hertzog ze O?sterreich, ze Steyr, ze Kernden und ze Krain, graf ze Tyrol et?u0020cetera, mbieten den erbern, geist-'],
  ['lichen, unsern lieben, andechtigen, dem provincial . . dem custer dem sichter und allen andern geistlichen amptluten minner'],
  ['bruder ordens, die des klosters ze Kunigsveld gewaltig sind, gegenwurtigen und kumftigen, den diser brief getzaigt wirt, uns'],
  ['gnad und alles gu?t. . wir bitten er alle und ewer yeglichen besunder und begern auch ernstlich, daz ir erun fleiz und ernste'],
  ['dartzu keret, damit das egenanten kloster unster stift in geistlicher zu?chtr, ordnung und fride beleiben und bestande welhe kloster-'],
  ['frowen daselbs sich aber dawider mit frevel, setzten und darinn nicht gehorsam sein wolten, daz ir die denn straffet nach eiers'],
  ['ordens gesetz und rechten und dawan derselben klosterfrowen adel oder wer frunde nicht schonet. . davon emphelhen wir'],
  ['dem edeln, unserm lieben oheini, gras Hannsen v

In [52]:
# Get GT transcriptions
page_path = re.sub(r'\..+$', r'.xml', str(img_path))
samples = charters_htr.ChartersDataset.extract_lines_from_pagexml(page_path)
[ s['transcription'] for s in samples]

['Wir, Leupolt, von gots gnaden hertzog ze Oͤsterreich, ze Steyr, ze Keͣrnden und ze Krain, graf ze Tyrol et\\u0020cetera, ✳ embieten den erbern, geist¬',
 'lichen, ùnsern lieben, andêchtigen, ✳ dem provincial, ✳ dem custer, ✳ dem bichter und allen andern geistlichen amptlùten, Minner¬',
 'bruder ordens, die des klosters ze Kùnigsveld gewaltig sind, gegenwùrtigen und kùmftigen, den diser brief getzaigt wirt, ùnser',
 'gnad und alles guͤt. ✳ wir bitten ew alle und ewr yeglichen besunder und begern auch ernstlich, daz ir ewern fleizz und ernste',
 'dartzu keret, damit das egenante kloster, unserr stift, in geistlicher zucht, ordnung und fride beleibe und bestande. ✳ welhe kloster¬',
 'frowen daselbs sich aber dawider mit freͣvel setzten und darinn nicht gehorsam sein wolten, daz ir die denn straffet nach ewers',
 'ordens gesetz und rechten ✳ und daran derselben klosterfrowen adel oder irer freunde nicht schonet. ✳ davon emphelhen wir',
 'dem edeln, unserm lieben oͤheim, graf Hannsen von 